In [1]:
# !pip install open3d
import open3d as o3d
import numpy as np
import sys

sys.path.append('./data/')
from camera import *
from dataloader import *

sys.path.append('./algorithm/')
from measurement import *
from pose import *
from update import *
from prediction import *

from pipeline import *

import matplotlib.pyplot as plt
import time

In [2]:
kf = KinectFusion()

KinectFusion pipeline total frame number: 798


In [4]:
kf.filter_depths()

Already filtered, break.


In [7]:
kf.depth_filtered

True

In [4]:
kf.first_frame()
timer = PerfTimer()

while kf.has_next_frame():
    timer.startMeasure("Overall frame")
    kf.next_frame()
    timer.stopMeasure()

KinectFusion pipeline total frame number: 798
Running frame: 0


KeyboardInterrupt: 